In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 7.2 MB/s 
     |████████████████████████████████| 448 kB 51.1 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json
import numpy as np
from konlpy.tag import Okt 
from collections import Counter
import re

import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
musinsa = {
           'total_reivew':[],
}

In [ ]:
total_page = 12951//60 +1

In [ ]:
total_page

216

# 큐레이팅 내용 추출

In [ ]:
for page in tqdm(range(1, total_page +1)):

    url = 'https://www.musinsa.com/app/musinsa/lists'
    

    params = {
        'sort':'new',
        'display_cnt':'60',
        'page': str(page),
        'total':'12951'

    }

    headers = {
              'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
              'referer': 'https://www.musinsa.com/app/musinsa/lists'
          }

    resp = requests.get(url, params = params, headers = headers)
    soup = BeautifulSoup(resp.content, 'lxml')
    total_reivew = soup.select('#productBox > li > div > div.musinsa_info.article_info > p.list_txt')
  
    for t in total_reivew:
        musinsa['total_reivew'].append(t.text.split(' / ')[0])

  0%|          | 0/216 [00:00<?, ?it/s]

In [ ]:
len(musinsa['total_reivew'])

0

In [ ]:
df = pd.DataFrame(musinsa)

In [ ]:
df

,total_reivew


In [ ]:
df.to_csv('큐레이팅 전체 확인.csv')

# 토큰화 & counter

In [ ]:
counting = {
           'word':[],
           'count':[]
}

In [ ]:
f = open('/content/큐레이팅 전체 확인.csv', 'r')

okt = Okt()

text = f.read()
noun = okt.nouns(text)
word_list = noun

word_count = {}
word_list

for word in word_list:
    word_count[word] = word_count.get(word, 0)+1
    keys = sorted(word_count.keys())

for word in keys:
    word + ':' + str(word_count[word])
    counting['word'].append(word)
    counting['count'].append(str(word_count[word]))


In [ ]:
df = pd.DataFrame(counting)

# 큐레이팅 페이지번호, 날짜, 내용 concat

In [ ]:
df_1 = pd.read_csv('/content/큐레이팅번호,날짜', index_col= 0)
df_2 = pd.read_csv('/content/큐레이팅 전체 확인.csv',index_col= 0)

In [ ]:
good = pd.concat([df_2,df_1], axis = 1)
good

In [ ]:
def apply_regular_expression(text):
    hangul = re.compile('[^ㄱ-ㅣ 가-힣]')
    result = hangul.sub('',text)
    return result

In [ ]:
good['total_reivew'] = good['total_reivew'].apply(apply_regular_expression)

In [ ]:
good

,total_reivew,Curating_Num,Date
0,이번 시즌 귀여운 분위기로 나를 가득 채워보고 싶다면 빈폴 멘이 피너츠와 함께 완성...,23734,2022.02.25
1,신선한 디자인의 티셔츠를 찾고 있다면 마켓만 한 게 없지 위트 있는 바트 심슨 캐릭...,23733,2022.02.25
2,사계절 내내 입기 좋은 팔렛의 아노락 재킷 미니멀한 카라 집업 디자인에 레이어드하기...,23732,2022.02.25
3,레트로한 리복 로고가 멋스러운 로얄 울트라를 소개한다 베이식한 실루엣에 사이드에 리...,23731,2022.02.25
4,봄을 맞이해 가볍게 들기 좋은 숄더백을 찾는다면 오스트카카의 누메르 필로우 백을 주...,23730,2022.02.25
...,...,...,...
12941,무신사와 알파 인더스트리 가 컬래버레이션을 통해 선보이는 재킷 알파 인더스트리의 ...,809,2015.11.16
12942,무신사에서만 만나볼 수 있는 르노이드의 스니커즈 코너 깔끔하다 직선적인 디자인으로 ...,752,2015.10.30
12943,스웰맙이 여성용으로 만든 스웨스셔츠 순면 소재 스웨트셔츠다 색도 순면처럼 자연에 가...,740,2015.10.27
12944,로얄위 의 청록색 스웨트셔츠 그 누가 청록색을 비단 여름의 색으로만 국한시켰던가 ...,730,2015.10.23


In [ ]:
good_df = good.drop_duplicates('total_reivew')

In [ ]:
good_df

,total_reivew,Curating_Num,Date
0,이번 시즌 귀여운 분위기로 나를 가득 채워보고 싶다면 빈폴 멘이 피너츠와 함께 완성...,23734,2022.02.25
1,신선한 디자인의 티셔츠를 찾고 있다면 마켓만 한 게 없지 위트 있는 바트 심슨 캐릭...,23733,2022.02.25
2,사계절 내내 입기 좋은 팔렛의 아노락 재킷 미니멀한 카라 집업 디자인에 레이어드하기...,23732,2022.02.25
3,레트로한 리복 로고가 멋스러운 로얄 울트라를 소개한다 베이식한 실루엣에 사이드에 리...,23731,2022.02.25
4,봄을 맞이해 가볍게 들기 좋은 숄더백을 찾는다면 오스트카카의 누메르 필로우 백을 주...,23730,2022.02.25
...,...,...,...
12941,무신사와 알파 인더스트리 가 컬래버레이션을 통해 선보이는 재킷 알파 인더스트리의 ...,809,2015.11.16
12942,무신사에서만 만나볼 수 있는 르노이드의 스니커즈 코너 깔끔하다 직선적인 디자인으로 ...,752,2015.10.30
12943,스웰맙이 여성용으로 만든 스웨스셔츠 순면 소재 스웨트셔츠다 색도 순면처럼 자연에 가...,740,2015.10.27
12944,로얄위 의 청록색 스웨트셔츠 그 누가 청록색을 비단 여름의 색으로만 국한시켰던가 ...,730,2015.10.23


In [ ]:
okt = Okt()

In [ ]:
new_dic = []
for code in tqdm(good['total_reivew']):
  new_dic.append(okt.nouns(code))

  0%|          | 0/12946 [00:00<?, ?it/s]

In [ ]:
for i in range(len(new_dic)):
    print(new_dic[i])

In [ ]:
words_list = []
for words in new_dic:
    words_list.append(','.join(words))

In [ ]:
len(words_list)

12946

In [ ]:
good['words'] = words_list

In [ ]:
good

,total_reivew,Curating_Num,Date,words
0,"이번 시즌, 귀여운 분위기로 나를 가득 채워보고 싶다면 빈폴 멘이 피너츠와 함께 완...",23734,2022.02.25,"이번,시즌,분위기,가득,보고,빈폴,피너츠,완성,빈티,아트워크,스웨,트,컬렉션,코튼,..."
1,신선한 디자인의 티셔츠를 찾고 있다면 마켓만 한 게 없지. 위트 있는 바트 심슨 캐...,23733,2022.02.25,"디자인,티셔츠,마켓,게,위트,바트,심슨,캐릭터,프린팅,오버,핏,티셔츠,옐로,퍼플,등..."
2,사계절 내내 입기 좋은 팔렛의 아노락 재킷. 미니멀한 카라 집업 디자인에 레이어드하...,23732,2022.02.25,"사계절,내내,팔렛,아노락,재킷,미니,카라,집업,디자인,레이어,드하,기,루즈,핏,웨어..."
3,레트로한 리복 로고가 멋스러운 로얄 울트라를 소개한다. 베이식한 실루엣에 사이드에 ...,23731,2022.02.25,"레트로,리복,로고,멋,로얄,울트라,소개,베이식,실루엣,사이드,리복,로고,컬러풀,자리..."
4,봄을 맞이해 가볍게 들기 좋은 숄더백을 찾는다면 오스트카카의 누메르 필로우 백을 주...,23730,2022.02.25,"봄,맞이,숄더백,오스,트,카카,누,메르,필,로우,백,주목,미니,직사각형,셰이프,체인..."
...,...,...,...,...
12941,무신사(MUSINSA)와 알파 인더스트리(Alpha Industries)가 컬래버레...,809,2015.11.16,"사,알파,인더스,트리,컬래버레이션,통해,재킷,알파,인더스,트리,헤리티지,수,디자인,..."
12942,무신사에서만 만나볼 수 있는 르노이드(Lenoid)의 스니커즈 코너2(Corner2...,752,2015.10.30,"사,나볼,수,르노,이드,스니커즈,코너,직선,디자인,모던,함,끝,달린다,어퍼,블랙,화..."
12943,스웰맙(Swellmob)이 여성용으로 만든 스웨스셔츠. 순면 소재 스웨트셔츠다. 색...,740,2015.10.27,"스웰맙,성용,스,웨스,셔츠,순면,소재,스웨,트,셔츠,색도,순면,자연,래글런,소매,만..."
12944,로얄위(The Royal We)의 청록색 스웨트셔츠. 그 누가 청록색을 비단 여름의...,730,2015.10.23,"로얄,위,의,청록색,스웨,트,셔츠,그,누가,청록색,비단,여름,색,국한,청록색,물결,..."


In [ ]:
good.to_csv('큐레이팅 전체 확인_찐막.csv')